In [1]:
import numpy as np
import pandas as pd
import neuroseries as nts


import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 

import basefunction.assembly as assembly

import basefunction.bk_load as bk
import basefunction.anass_load as at

In [2]:
def binSpikes(neurons,binSize = 0.025,start = 0,stop = 0):
    if stop == 0:
        stop = np.max([neuron.as_units('s').index[-1] for neuron in neurons])
    bins = np.arange(start,stop,binSize)
    binned = []
    for neuron in neurons:
        hist,b = np.histogram(neuron.as_units('s').index,bins = bins)
        binned.append(hist)
    return np.array(binned),b

In [3]:
path = bk.get_session()['Path'][5]
bk.set_current_session(path)

neurons, metadata = bk.loadSpikeData(path)
neurons = {'all':neurons}
metadata = {'all':metadata}

structures = ['BLA','Hpc','CeCM']
shanks = {}
masks = {}
for stru in structures:
    shanks.update({stru:at.get_structure_shanks(bk.session,stru)})
    masks.update({stru:[i in shanks[stru] for i in metadata['all'][:,0]]})
    neurons.update({stru:neurons['all'][masks[stru]]})
    metadata.update({stru:metadata['all'][masks[stru]]})

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Loading Rat08-20130713.clu.1
Loading Rat08-20130713.clu.2
Loading Rat08-20130713.clu.3
Loading Rat08-20130713.clu.4
Loading Rat08-20130713.clu.5
Loading Rat08-20130713.clu.6
Loading Rat08-20130713.clu.7
Loading Rat08-20130713.clu.8
Loading Rat08-20130713.clu.9
Loading Rat08-20130713.clu.10
Loading Rat08-20130713.clu.11
Loading Rat08-20130713.clu.12
Loading Rat08-20130713.clu.13
Loading Rat08-20130713.clu.14
Loading Rat08-20130713.clu.15
Loading Rat08-20130713.clu.16
Loading Rat08-20130713.clu.17
Loading Rat08-20130713.clu.18
Loading Rat08-20130713.clu.19
Loading Rat08-20130713.clu.20


In [32]:
act_bla,b = binSpikes(neurons['BLA'],binSize=0.025)
plt.figure()
sns.heatmap(act_bla[:,0:500],cmap='Reds') # shows first 50 time bins of simulation
plt.show()

In [33]:
# using circular shuffling for computing null distribution 
                    # (MUST USE IT IF YOU HAVE STRONG AUTOCORRELATIONS)

nshu = 1000 # defines number of controls to run 
#                     (i.e., number of elements in null hypothesis distribution)

percentile = 99.5 # defines which percentile of control distribution
#                                        will be used as significance threshold


t0 = time.time()
patterns, significance, zactmat = assembly.runPatterns(act_bla,
                                                       nullhyp='mp',
                                                       nshu = nshu,
                                                       percentile = percentile)

tf = np.round(1000*(time.time()-t0))/1000



Text(0.5, 0.98, 'Using circular shuffling, elapsed time: 31.543 seconds')

In [41]:
plt.figure(figsize=(15,5))
for (pi,pattern) in enumerate(patterns[0:7]):
        plt.subplot(1,len(patterns[0:7]),1+pi)
        plt.stem(pattern,use_line_collection = True)
        plt.xlabel('neuron #')
        plt.ylabel('weight')
plt.suptitle('Using circular shuffling, elapsed time: '+str(tf)+' seconds')

Text(0.5, 0.98, 'Using circular shuffling, elapsed time: 31.543 seconds')

In [59]:
plt.figure(figsize=(15,6))
s1 = plt.subplot(211)
plt.imshow(act_bla[:,0:150000],cmap='Reds',interpolation='nearest',aspect='auto')
plt.title('z-scored activity matrix')
plt.ylabel('neuron #')

plt.subplot(212,sharex=s1)
plt.plot(assemblyAct[:,0:150000].T,linewidth=1)
plt.title('assembly activities')
plt.xlim(0,14999)
plt.xlabel('time bin')
plt.ylabel('strength')
plt.tight_layout()
plt.show()

In [62]:
import scipy
actmat,b = binSpikes(neurons['BLA'],binSize=0.01)
zactmat = scipy.stats.zscore(actmat,1)
assemblyAct = assembly.computeAssemblyActivity(patterns,zactmat)

for i,pattern in enumerate(patterns):
    members = np.where(np.abs(pattern)>0.15)
    print(members)
    peak_ = np.argmax(assemblyAct[i])
    prange = peak_ + np.array([-1000,1000])
    smps2plot = np.arange(prange[0],prange[1])
    plt.figure()
    bk.rasterPlot(neurons['BLA'],np.array([prange*0.01]),col = 'black')
    bk.rasterPlot(neurons['BLA'][members],np.array([prange*0.01]),col = 'red')
    plt.plot(smps2plot*0.01,assemblyAct[i][smps2plot]/20)
    plt.grid()
    plt.show()

(array([69, 84, 86, 93], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([23, 62, 70, 71, 78, 80], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([ 9, 13, 15, 17, 20], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([ 4, 35, 39, 45, 52, 54, 58], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([ 6, 31, 48, 53, 63, 73, 78, 82, 86, 89], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([40, 45, 46, 50, 53, 62, 68], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([ 3, 10, 11, 12, 24, 25, 50, 58, 63, 78], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([16, 19, 23, 59, 60, 61, 73, 83, 90], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([ 8, 22, 50, 60, 66, 67, 76], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(array([68, 82, 86, 87, 90], dtype=int64),)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


<class 'numpy.ndarray'>
(array([13, 22, 45, 55, 59, 64, 83, 86], dtype=int64),)
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


<class 'numpy.ndarray'>
(array([ 2,  7, 11, 19, 41, 44], dtype=int64),)
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


<class 'numpy.ndarray'>
(array([ 6, 22, 37, 56, 59, 62, 66, 67, 73], dtype=int64),)
<class 'numpy.ndarray'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  import sys


<class 'numpy.ndarray'>


In [53]:
a = np.in1d([1,2,11,3],[0,1,2,3,4,5,6,7])

In [54]:
a

array([ True,  True, False,  True])